# Dataset Generation using Emulator

Start the emulator and save a screenshot containing the screen of the emulated device exactly.
Save the path to the screenshot in the variable below:

In [ ]:
take_screenshot = False
use_android_screenshot = False
get_image_from_server = True
# TODO use_camera?

screenshot = 'AndroidScreenshot.png'

Run the cell below and check if the output matches the screen of the emulated device.

In [ ]:
if(take_screenshot):
    import pyautogui

    box = pyautogui.locateOnScreen(screenshot, grayscale=True)

    location = [box.left, box.top, box.width, box.height]

    im = pyautogui.screenshot(region=location)
    display(im)

Use the next cell to adjust the screenshot location and size if necessary.

In [ ]:
if(take_screenshot):
    location[3] = location[3] + 1

    im = pyautogui.screenshot(region=location)
    print(location)
    display(im)

Configure the parameters in the next cell before starting the dataset collection.

In [ ]:
# Number of elements that will be collected.
DATASET_SIZE = 200
# Path of directory at which the dataset will be saved.
DATASET_PATH = "dataset/"
# IP adress of device running mobile application.
APPLICATION_HOST = "192.168.1.100"
# Port on which the server of the application is listening.
APPLICATION_PORT = 1280
# IP adress of device capturing images.
CAMERA_HOST = "localhost"
# Port on which the server for capturing is listening.
CAMERA_PORT = 1300

Now, run the cell below. It will start a server, which the android application can then connect to and upload it's data. Thus, you can start the android application after starting the next cell. The dataset collection will happen automatically.

In [ ]:
import json
import socket
import socketserver
import os
import base64
from io import BytesIO
from PIL import Image
from IPython.display import clear_output
import time

class Dataset:
    def __init__(self, size, path):
        self.total_size = size
        self.current_size = 0
        self.path = path
        # TODO: check path for validity and if dir is empty.
        
    def captureAndSaveImage(self, path: str):
        im = pyautogui.screenshot(path, region=location)
        # display(im)
            
    def getImageFromServer(self):
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        client.connect((CAMERA_HOST, CAMERA_PORT))
        request = {"header": "image"}
        client.sendall((json.dumps(request) + "\n").encode())
        response = client.makefile().readline()
        return response
            
    def getMaskFromServer(self):
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        client.connect((CAMERA_HOST, CAMERA_PORT))
        request = {"header": "mask"}
        client.sendall((json.dumps(request) + "\n").encode())
        response = client.makefile().readline()
        return response
        
    def getFromServerAndSaveImage(self, path: str):
        image_base64 = self.getImageFromServer()
        image_bytes = base64.b64decode(image_base64)
        image_file = BytesIO(image_bytes)
        image = Image.open(image_file)
        display(image)
        image.save(path)
        
    def getFromServerAndSaveMask(self, path: str):
        image_base64 = self.getMaskFromServer()
        image_bytes = base64.b64decode(image_base64)
        image_file = BytesIO(image_bytes)
        image = Image.open(image_file)
        display(image)
        image.save(path)
        
    def saveAnnotations(self, path: str, data):
        
        data.pop("-1", None)
        for id_of_element, interactive_element in data.items():
            data.get(id_of_element).pop("mask", None)
        try:
            with open(path, "w") as file:
                file.write(json.dumps(data, indent=2))
        except Exception as e:
            print(e.message, e.args)
        
    def saveMasks(self, data):
        data = json.loads(data)
        for id_of_element, interactive_element in data.items():
            mask_bytes = base64.b64decode(interactive_element.get("mask"))
            mask_file = BytesIO(mask_bytes)
            mask = Image.open(mask_file)
            #display(mask)
            if id_of_element == "-1":
                if use_android_screenshot:
                    mask.save(self.path + f"{self.current_size}.png")
            else:
                mask.save(self.path + f"{self.current_size}_{id_of_element}.png")
            
            
    def handleCollection(self, data) -> bool:
        self.saveMasks(data)
        self.saveAnnotations(self.path + f"{self.current_size}.json", json.loads(data))
        self.current_size = self.current_size + 1
        print(f"Progress: {self.current_size}/{self.total_size}")
        if self.current_size >= self.total_size:
            print("Dataset collected.")
            return True
        return False


dataset = Dataset(DATASET_SIZE, DATASET_PATH)

In [ ]:
class TCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        #print("Client connected.")
        # clear_output(wait=True)
        try:
            data = self.rfile.readline().strip()
            
            if json.loads(data).get("answer") == "ready":
                #print("received ready")
                if(take_screenshot):
                    dataset.captureAndSaveImage(dataset.path + f"{dataset.current_size}.png")
                elif(get_image_from_server):
                    dataset.getFromServerAndSaveImage(dataset.path + f"{dataset.current_size}.png")

                response = {"answer": "masks"}
                self.wfile.write(json.dumps(response).encode("utf-8"))
                self.wfile.write(b"\n")
                # print("send masks")

                data = self.rfile.readline().strip()
                # print("received data")

                collectionFinished = dataset.handleCollection(data)

                if collectionFinished:
                    self.server._BaseServer__shutdown_request = True

            response = {"answer": "randomize"}
            self.wfile.write(json.dumps(response).encode())
        except Exception as e:
            print(e.message, e.args)

with socketserver.TCPServer((HOST, PORT), TCPHandler) as server:
    print("Starting server.")
    server.serve_forever()

In [ ]:
dataset.getFromServerAndSaveImage(dataset.path + f"{dataset.current_size}.png")

In [ ]:
import socket
import json
import time

def makeRequest(header):
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((APPLICATION_HOST, APPLICATION_PORT))

    request = {"header": header}
    client.sendall((json.dumps(request) + "\n").encode())

    response = json.loads(client.makefile().readline())
    print(json.dumps(response, indent = 2))

    client.close()

    return response

while(dataset.current_size < dataset.total_size):
    response = makeRequest("ready?")
    if(response["header"] != "ready!"):
        continue

    time.sleep(0.2)
    dataset.getFromServerAndSaveImage(dataset.path + f"{dataset.current_size}.png")

    response = makeRequest("masks?")
    if(response["header"] != "masks!"):
        continue

    data = response["body"]
    data.pop("-1", None)

    try:
        with open(dataset.path + f"{dataset.current_size}.json", "w") as file:
            file.write(json.dumps(data, indent=2))
    except Exception as e:
        print(e.message, e.args)

    for mask_index in data:
        response = makeRequest("next state?")
        time.sleep(0.2)
        dataset.getFromServerAndSaveMask(dataset.path + f"{dataset.current_size}_{mask_index}.png")

    response = makeRequest("randomize")

    dataset.current_size = dataset.current_size + 1